In [1]:
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
from prep_data import *
import locale
# locale.setlocale(locale.LC_TIME, 'tr_TR.UTF8')
# 

# Sales

In [ ]:
df_totals_total, df_totals_cities, df_granular, df_granular_cities = sales_by_cities_df()

In [ ]:
df_totals_cities

In [ ]:
df_totals_cities.columns


In [ ]:
df_granular

In [ ]:

# def total_sales_plot(df_totals):

fig = go.Figure(data=[go.Line(name=city, x=df_totals_cities.index, y=df_totals_cities[city]) for city in df_totals_cities.columns], 
                layout=dict(height=800, width=1800))
fig.update_layout(title="İllere Göre Konut Satışı (Yıl Bazında)", xaxis_title="Yıl", yaxis_title="Toplam Konut Satışı", barmode='stack')
fig.update_xaxes(tickmode='linear')
fig.update_yaxes(tickmode='linear')

fig.show()

In [ ]:

# def total_sales_plot(df_totals):

fig = go.Figure(data=[go.Bar(name="Adana", x=df_totals_cities.index, y=df_totals_cities["1 Adana"])], 
                layout=dict(height=800, width=1800))
fig.update_layout(title="İllere Göre Konut Satışı (Yıl Bazında)", xaxis_title="Yıl", yaxis_title="Toplam Konut Satışı", barmode='stack')
fig.update_xaxes(tickmode='linear')
fig.update_yaxes(tickmode='linear')

fig.show()

In [ ]:
df_granular_cities = df_granular.drop("Total", axis=1)
df_granular_cities.head()

In [ ]:

# def total_sales_plot(df_totals):

fig = go.Figure(data=[go.Line(name=city, x=df_granular_cities.index, y=df_granular_cities[city]) for city in df_totals_cities.columns], 
                layout=dict(height=800, width=1800))
fig.update_layout(title="İllere Göre Konut Satışı (Ay bazında)", xaxis_title="Yıl", yaxis_title="Toplam Konut Satışı", barmode='stack')

fig.show()

In [ ]:
df_f = pd.read_excel("./datasets/İllere göre yabancılara konut satış sayısı.xls")
df_f.rename(columns={'Unnamed: 1': 'Şehir'}, inplace=True)
df_f.tail(14)

df_f_total = df_f[df_f['Şehir'] == "Toplam - Total"].drop(["Şehir", "Toplam"], axis=1)#.set_index("Yıl")
df_f_total["Yıl"] = df_f_total["Yıl"].astype(int)
# df_f_total.index = df_f_total.index.astype(int)

df_f["Yıl"] = df_f["Yıl"].ffill().astype(int)
df_f.drop("Toplam", axis=1, inplace=True)
df_f.tail(19)

In [ ]:
df_f.tail()

In [ ]:
df_f_total.tail()


In [ ]:
# Fix the 2024 data
# Reshape the DataFrame into long format
df_long = df_f_total.melt(id_vars=['Yıl'], var_name='Ay', value_name='Sayı')

# Aggregate the data by year, month, and city
df_f_total_aggregated = df_long.groupby(['Yıl', 'Ay']).sum().reset_index()

# Create a mapping of month names to their corresponding numbers
month_mapping = {
    'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
    'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
}

# Convert 'Yıl' and 'Ay' to datetime format
df_f_total_aggregated['Tarih'] = df_f_total_aggregated.apply(lambda row: pd.Timestamp(int(row['Yıl']), month_mapping[row['Ay']], 1), axis=1)

# Drop the 'Yıl' and 'Ay' columns if not needed anymore
df_f_total_aggregated.drop(columns=['Yıl', 'Ay'], inplace=True)

df_f_total_aggregated = df_f_total_aggregated.sort_values(by='Tarih')
df_f_total_aggregated.set_index("Tarih", inplace=True)
df_f_total_aggregated["Sayı"] = df_f_total_aggregated["Sayı"].astype(int)
df_f_total_aggregated.info()

In [ ]:
df_f_total_aggregated.tail(15)

In [ ]:
df_f_cities = df_f[df_f['Şehir'] != "Toplam - Total"]
df_f_cities.head()

In [ ]:
# Fix the 2024 data
# Reshape the DataFrame into long format
df_long = df_f_cities.melt(id_vars=['Yıl', 'Şehir'], var_name='Ay', value_name='Sayı')

# Aggregate the data by year, month, and city
df_f_cities_aggregated = df_long.groupby(['Yıl', 'Şehir', 'Ay']).sum().reset_index()

# Create a mapping of month names to their corresponding numbers
month_mapping = {
    'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
    'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
}

# Convert 'Yıl' and 'Ay' to datetime format
df_f_cities_aggregated['Tarih'] = df_f_cities_aggregated.apply(lambda row: pd.Timestamp(int(row['Yıl']), month_mapping[row['Ay']], 1), axis=1)

# Drop the 'Yıl' and 'Ay' columns if not needed anymore
df_f_cities_aggregated.drop(columns=['Yıl', 'Ay'], inplace=True)


df_f_cities_aggregated = df_f_cities_aggregated.sort_values(by='Tarih')
df_f_cities_aggregated.set_index("Tarih", inplace=True)
df_f_cities_aggregated["Sayı"] = df_f_cities_aggregated["Sayı"].astype(int)
df_f_cities_aggregated.info()

In [ ]:
df_f_cities_aggregated.head(20)

In [2]:

df_f_total_aggregated, df_f_cities_aggregated = sales_by_cities_foreigners_df()

In [3]:
df_f_cities_aggregated


,Şehir,Total
Tarih,,
2013-01-01,48,75
2013-01-01,33,45
2013-01-01,35,21
2013-01-01,16,23
2013-01-01,99,23
...,...,...
2024-04-01,16,28
2024-04-01,34,454
2024-04-01,77,23


In [7]:
df_f_cities_aggregated["Şehir"].unique()

array([48, 33, 35, 16, 99,  9,  7, 54,  6, 34, 77, 61, 55, 41],
      dtype=int64)

In [22]:
city_code_map = {0: 'ÜLKE',
 1: 'ADANA',
 2: 'ADIYAMAN',
 3: 'AFYONKARAHİSAR',
 4: 'AĞRI',
 5: 'AMASYA',
 6: 'ANKARA',
 7: 'ANTALYA',
 8: 'ARTVİN',
 9: 'AYDIN',
 10: 'BALIKESİR',
 11: 'BİLECİK',
 12: 'BİNGÖL',
 13: 'BİTLİS',
 14: 'BOLU',
 15: 'BURDUR',
 16: 'BURSA',
 17: 'ÇANAKKALE',
 18: 'ÇANKIRI',
 19: 'ÇORUM',
 20: 'DENİZLİ',
 21: 'DİYARBAKIR',
 22: 'EDİRNE',
 23: 'ELAZIĞ',
 24: 'ERZİNCAN',
 25: 'ERZURUM',
 26: 'ESKİŞEHİR',
 27: 'GAZİANTEP',
 28: 'GİRESUN',
 29: 'GÜMÜŞHANE',
 30: 'HAKKARİ',
 31: 'HATAY',
 32: 'ISPARTA',
 33: 'MERSİN',
 34: 'İSTANBUL',
 35: 'İZMİR',
 36: 'KARS',
 37: 'KASTAMONU',
 38: 'KAYSERİ',
 39: 'KIRKLARELİ',
 40: 'KIRŞEHİR',
 41: 'KOCAELİ',
 42: 'KONYA',
 43: 'KÜTAHYA',
 44: 'MALATYA',
 45: 'MANİSA',
 46: 'KAHRAMANMARAŞ',
 47: 'MARDİN',
 48: 'MUĞLA',
 49: 'MUŞ',
 50: 'NEVŞEHİR',
 51: 'NİĞDE',
 52: 'ORDU',
 53: 'RİZE',
 54: 'SAKARYA',
 55: 'SAMSUN',
 56: 'SİİRT',
 57: 'SİNOP',
 58: 'SİVAS',
 59: 'TEKİRDAĞ',
 60: 'TOKAT',
 61: 'TRABZON',
 62: 'TUNCELİ',
 63: 'ŞANLIURFA',
 64: 'UŞAK',
 65: 'VAN',
 66: 'YOZGAT',
 67: 'ZONGULDAK',
 68: 'AKSARAY',
 69: 'BAYBURT',
 70: 'KARAMAN',
 71: 'KIRIKKALE',
 72: 'BATMAN',
 73: 'ŞIRNAK',
 74: 'BARTIN',
 75: 'ARDAHAN',
 76: 'IĞDIR',
 77: 'YALOVA',
 78: 'KARABÜK',
 79: 'KİLİS',
 80: 'OSMANİYE',
 81: 'DÜZCE',
 99: 'Diğer iller - Other Provinces'}


city_code_f = [48, 33, 35, 16, 99,  9,  7, 54,  6, 34, 77, 61, 55, 41]

fig = go.Figure()
city_code = 7
try:
    if city_code is None:
        fig = px.line(df_f_cities_aggregated, x=df_f_cities_aggregated.index, 
                        y='Total', color='Şehir', title='İllere Göre Yabancılara Konut Satışı (Ay Bazında)')
    else:
        p_city_code = city_code
        if city_code not in city_code_f:
            city_code = 99
        dff = df_f_cities_aggregated[df_f_cities_aggregated['Şehir'] == city_code]
        
        fig = px.line(dff, x=dff.index, y='Total', color='Şehir', title=f'Yabancılara Konut Satışı (Ay Bazında): {city_code_map[p_city_code].capitalize()}',
                        color_discrete_sequence=px.colors.qualitative.Vivid)
        fig.update_layout(showlegend=False)
    fig.update_layout(width=1800, height=800)
except Exception as e:
    print(f"Error in plot function: {e}")
    raise

fig.show()

# Population

In [2]:
def population_df(df_p):    
    df_p = pd.read_excel("./datasets/favori_raporlar.xlsx",  sheet_name="MAHALLE NÜFUSU", index_col=0)
    cols = ['il kayit no',
    'ilçe kayit no',
    'belde/köy kayit no',
    'mahalle kayit no',
    'il adi',
    'ilçe adi',
    'belediye adi',
    'mahalle adi',
    'mahallenin bağli olduğu belediyenin niteliği',
    'toplam',
    'erkek',
    'kadin']
    df_p.columns = cols
    df_p_cities = df_p.drop(["toplam", "mahallenin bağli olduğu belediyenin niteliği", "belediye adi"], axis=1)
    # df_p_cities[(df_p_cities["mahalle adi"]==" ".join("2000 Evler Mh.".split()[:-1]).upper()) & (df_p_cities["ilçe adi"]=="seyhan".upper())] #for query
    df_p = df_p[["il adi", "ilçe adi", "mahalle adi", "erkek", "kadin"]]

    df_p.erkek = df_p.erkek.replace("C", "0").astype(int)
    df_p.kadin = df_p.kadin.replace("C", "0")
    df_p.kadin = df_p.kadin.replace("-", "0").astype(int)

    df_p = df_p.groupby(["il adi", "ilçe adi", "mahalle adi"])[["erkek", "kadin"]].sum().reset_index()

    return df_p



In [2]:
df_p = population_df()
df_p

,il adi,ilçe adi,mahalle adi,erkek,kadin
0,ADANA,ALADAĞ,AKPINAR,219,221
1,ADANA,ALADAĞ,AKÖREN,456,437
2,ADANA,ALADAĞ,BAŞPINAR,393,369
3,ADANA,ALADAĞ,BOZTAHTA,118,120
4,ADANA,ALADAĞ,BÜYÜKSOFULU,872,803
...,...,...,...,...,...
31797,ŞIRNAK,İDİL,TEMEL,971,1010
31798,ŞIRNAK,İDİL,TURGUT ÖZAL,3919,3872
31799,ŞIRNAK,İDİL,YENİ,5153,5044
31800,ŞIRNAK,İDİL,YUKARI,2551,2569


In [140]:
df_p_il = df_p.groupby(["il adi"])[["erkek", "kadin"]].sum().reset_index()
df_p_il

,il adi,erkek,kadin
0,ADANA,1130862,1132511
1,ADIYAMAN,239832,238201
2,AFYONKARAHİSAR,289709,291269
3,AKSARAY,175485,175294
4,AMASYA,125363,126696
...,...,...,...
76,ÇORUM,194721,202925
77,İSTANBUL,7933674,7907212
78,İZMİR,2199264,2226499
79,ŞANLIURFA,1079015,1064005


In [ ]:
# Pie chart
# choloropleth

In [ ]:
# import plotly.graph_objects as go

# # Sample data for the pie chart
# labels = ['Erkek', 'Kadın']
# values = [df_p.iloc[0, 1], df_p.iloc[0, 2]]
# colors = ['skyblue', 'salmon']

# # Create the Pie chart with custom colors and slice pulled out
# fig = go.Figure(go.Pie(
#     labels=labels,
#     values=values,
#     marker=dict(colors=colors),
#     hovertemplate='<b>%{label}</b><br>%{value}'
# ), layout=dict(width=400, height=400))

# # Update the name of the trace
# fig.data[0].name = ''

# # Set the title
# fig.update_layout(title='Cinsiyet Dağılımı')

# # Show the figure
# fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def population_plot(df_p, city_code=0):
    # Create the figure with subplots
    fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}]], subplot_titles=(df_p.iloc[city_code, 0].capitalize(), "İl ve İlçe Merkezleri", "Belde ve Köyler"))

    # Define the data for each pie chart
    values1 = [df_p.iloc[city_code, 2], df_p.iloc[city_code, 3]]
    values2 = [df_p.iloc[city_code, 5], df_p.iloc[city_code, 6]]
    values3 = [df_p.iloc[city_code, 8], df_p.iloc[city_code, 9]]
    labels = ['Erkek', 'Kadın']
    colors = ['skyblue', 'salmon']

    # Add the first pie chart
    fig.add_trace(go.Pie(
        name="",
        labels=labels,
        values=values1,
        marker=dict(colors=colors),
        hovertemplate='<b>%{label}</b><br>%{value}',
        pull=[0, 0],
        textfont=dict(size=16, family="Balto", color="black"),  # Adjust text size and color
        textinfo='label+percent',  # Show labels and percentages
    ), row=1, col=1)

    # Add the second pie chart
    fig.add_trace(go.Pie(
        name="",
        labels=labels,
        values=values2,
        marker=dict(colors=colors),
        hovertemplate='<b>%{label}</b><br>%{value}',
        pull=[0, 0],
        textfont=dict(size=16, family="Balto", color="black"),  # Adjust text size and color
        textinfo='label+percent',  # Show labels and percentages
    ), row=1, col=2)

    # Add the third pie chart
    fig.add_trace(go.Pie(
        name="",
        labels=labels,
        values=values3,
        marker=dict(colors=colors),
        hovertemplate='<b>%{label}</b><br>%{value}',
        pull=[0, 0],
        textfont=dict(size=16, family="Balto", color="black"),  # Adjust text size and color
        textinfo='label+percent',  # Show labels and percentages
    ), row=1, col=3)

    # Update layout
    fig.update_layout(
        title='Cinsiyet Dağılımları',
        width=1100,
        height=500,
        title_font=dict(size=30, family="Balto", ),  # Adjust title font
        title_pad_b=10,
        font=dict(
            family="Balto",  # Adjust default font family for labels
            size=14,  # Adjust default font size for labels
            color="black"  # Adjust default font color for labels
        ),
        legend=dict(
            title_font=dict(size=18, family="Balto"),  # Adjust legend title font
            font=dict(size=16, family="Balto")  # Adjust legend item font
        )
    )

    # Update subplot titles font and style

    for annotation in fig['layout']['annotations']:
        annotation['y'] = 0.95  # Adjust this value for more/less padding

        # Add the text annotation below the plots
    fig.add_annotation(
    text=f"Toplam Nüfus: <b>{df_p.iloc[city_code, 1]}</b>",
    xref="paper", yref="paper",
    x=0.5, y=-0.2,
    showarrow=False,
    font=dict(size=18, family="Balto", color="black"),
    align="center"
)
    return fig
fig = population_plot(df_p=df_p, city_code=1)
# Show the figure
fig.show()


In [ ]:
import plotly.express as px

# Assuming you have the GeoJSON file for Turkey provinces
geojson_file = './tr-cities.json'

# Calculate total population
df_p_cities['total_population'] = df_p_cities['toplam-erkek'] + df_p_cities['toplam-kadin']

# Load GeoJSON data
import json
with open(geojson_file) as f:
    turkey_geojson = json.load(f)

# Ensure the province names in the DataFrame match the names in the GeoJSON file
# This might require some preprocessing if the names don't match exactly


# Create the choropleth map
fig = px.choropleth(
    df_p_cities,
    geojson=turkey_geojson,
    locations='il adi',
    featureidkey="properties.name",  # This should match the property name in your GeoJSON file for province names
    color='total_population',
    hover_name='il adi',
    hover_data={'toplam-erkek': True, 'toplam-kadin': True, 'total_population': True},
    color_continuous_scale='Viridis',
    labels={'total_population': 'Total Population'},
    title='Total Population by Province in Turkey'
)

# Update the layout for better visualization
fig.update_geos(
    visible=False,
    resolution=50,
    showcountries=True,
    countrycolor="Black",
    showcoastlines=True,
    coastlinecolor="Black",
    showland=True,
    landcolor="white",
)

fig.update_layout(
    title_font=dict(size=24, family="Arial, sans-serif"),
    font=dict(size=12, family="Arial, sans-serif"),
    margin={"r":0,"t":50,"l":0,"b":0}
)

# Show the map
# fig.show()


In [ ]:
turkey_geojson.keys()

In [ ]:
turkey_geojson["features"][2]["properties"]["name"]

In [ ]:
def total_sales_monthly_foreigners_plot(df_f_cities_aggregated, city=None):
    """
        City must be spelled properly
    """
    if city == None:
        fig = px.line(df_f_cities_aggregated, x=df_f_cities_aggregated.index, 
                      y='Total', color='Şehir', title='Real Estate Sold Over Time',
                      )
    else:
        city = city.capitalize()
        cities = df_f_cities_aggregated["Şehir"].unique()
        if city not in cities:
            city = 'Diğer iller - Other provinces'
        dff = df_f_cities_aggregated[df_f_cities_aggregated['Şehir'] == city]
        fig = px.line(dff, x=dff.index, y='Total', color='Şehir', title='Real Estate Sold Over Time',
                     color_discrete_sequence=px.colors.qualitative.Vivid)
    fig.update_layout(width=1800, height=800)
    return fig

fig = total_sales_monthly_foreigners_plot(df_f_cities_aggregated, "123")
fig.show()

In [109]:
import pandas as pd

data = {
    'city': ['CityA', 'CityA', 'CityA', 'CityB', 'CityB', 'CityB'],
    'town': ['Town1', 'Town1', 'Town2', 'Town1', 'Town1', 'Town2'],
    'district': ['District1', 'District1', 'District2', 'District3', 'District3', 'District4'],
    'district_id': [1, 2, 3, 4, 5, 6],
    'population_male': [500, 750, 1000, 1250, 1500, 1750],
    'population_female': [500, 750, 1000, 1250, 1500, 1750]
}

df = pd.DataFrame(data)
df

,city,town,district,district_id,population_male,population_female
0,CityA,Town1,District1,1,500,500
1,CityA,Town1,District1,2,750,750
2,CityA,Town2,District2,3,1000,1000
3,CityB,Town1,District3,4,1250,1250
4,CityB,Town1,District3,5,1500,1500
5,CityB,Town2,District4,6,1750,1750


In [111]:

town_aggregated = df.groupby(['city', 'town'])[['population_male', 'population_female']].sum().reset_index()

town_aggregated

,city,town,population_male,population_female
0,CityA,Town1,1250,1250
1,CityA,Town2,1000,1000
2,CityB,Town1,2750,2750
3,CityB,Town2,1750,1750
